# Notebook 2: A more complicated example

### Requirements:

- `numpy`
- `pandas`
- `matplotlib`
- `itertools`
- `scipy.stats`
- `numba`

In [1]:
import sys
sys.path.insert(0,"../")
# import
import battlesim as bsm

### Additional Imports

In [2]:
from IPython.display import HTML
%matplotlib inline

## A more complex battle

In [3]:
battle = bsm.Battle("../datasets/starwars-clonewars.csv")

## Initialise

50 Clone troopers versus 100 battledroids.

Except the clones will be in two army groups, and not one.

In [4]:
cols = {0:'red',1:'blue'}
labels = {0:'Republic',1:'CIS'}

In [5]:
# define 100 battledroids, 50 clones
battle.create_army([("B1 battledroid", 80), ("Clone Trooper", 20), ("Clone Trooper", 30)])

# battle droids near (0, 0), clones near (10, 10), default distribution is gaussian
battle.apply_position([
    dict(name="beta"),
    dict(name="gaussian", loc=10., scale=1.),
    dict(name="normal", y_loc=5., x_loc=7., scale=1.)
])

# set rolling ai, keep random init
battle.set_rolling_ai(["random","pack","random"])

bsm.Battle(init=True, n_armies=3, simulated=False)

## Run

Here we mostly use random AI selection strategies.

In [6]:
# run a battle
S1 = battle.simulate()

In [7]:
battle.sim_jupyter()

## Saving the animation to file

Rather than generate HTML, we can save the animation as a `.gif` rather easily:

In [14]:
battle.sim_export("../simulations/sim1.gif", writer="imagemagick")

## Simulating Multiple Runs at once

Whilst it is nice to see the animation play out in a single run, random effects within the simulation cause variations when you run them multiple times (i.e they are not *deterministic*). What is more interesting to see the average effects across **iterative runs**.

This is achieved using the `simulate_k()` function, also found as a function to our `battle` object. Here we can select `k` as a parameter (anything greater than 1).

In [15]:
S2 = battle.simulate_k(30)

### The output is different... what?

When using `simulate_k()`, the full output from every single `k` run is not returned because the dataset quickly explodes by $n$ and $k$. What is returned instead is, for every Allegiance/Team per $k$, the number of units alive/remaining. This then provides **just the result** of the battle and not the detailed nitty-gritty.

In [17]:
S2.head()

,Republic,CIS
0,49,0
1,49,0
2,50,0
3,49,0
4,48,0


In this case, the Republic win every single iteration. Clearly they have an advantage somehow (humans smarter than droids?).

In [18]:
S2.gt(0).sum()

Republic    30
CIS          0
dtype: int64

## Another example, different units

This time, we have 4 army groups, 3 CIS, 1 Republic, with different locations, `nearest` AI options for all, and 1000 iterations.

In [21]:
battle2 = bsm.Battle("../datasets/starwars-clonewars.csv")
battle2.create_army([("B1 battledroid", 20), ("Clone Trooper", 20),
                    ("B2 battledroid", 7), ("Magmaguard", 4)])
battle2.apply_position([bsm.Distribution("uniform",loc=2.,scale=2.),bsm.Distribution(scale=1.5),
                       bsm.Distribution(loc=-3.,scale=2.),bsm.Distribution(loc=4.,scale=2.).setx(loc=-1.)])
battle2.set_initial_ai(["nearest"]*4)
battle2.set_rolling_ai(["nearest"]*4)
# run a battle
S3 = battle2.simulate()

In [22]:
HTML(bsm.quiver_fight(S3,labels,cols).to_jshtml())

In [23]:
S4 = battle2.simulate_k(1000)
S4.gt(0).sum()

Republic    1000
CIS            0
dtype: int64

Here we see that the Republic has a 95% win rate compared to the droids, even though they are severely outnumbered.

This is the end to the second notebook.